In [38]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig

# Initialize the tokenizer
# tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096", do_lower_case=False)
# tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast = False)
# model = AutoModel.from_pretrained("bert-large-cased", output_hidden_states=True)

In [37]:
AutoTokenizer.from_pretrained?

In [2]:
tokenizer.tokenize('\u00a0')

[]

In [3]:
tokenizer.tokenize(' ')

[]

In [40]:
tokenizer.tokenize("Real estate Transactions are some of the higher value transactions\u00a0performed by individuals and organizations. ")

['Real',
 'estate',
 'Trans',
 '##actions',
 'are',
 'some',
 'of',
 'the',
 'higher',
 'value',
 'transactions',
 'performed',
 'by',
 'individuals',
 'and',
 'organizations',
 '.']

### New `transformers` does not `encode` as expected
#### The trick is to use `use_fast=False`

```
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer.tokenize("Real estate Transactions")
# ['Real', 'estate', 'Trans', '##actions']
tokenizer.encode(['Real', 'estate', 'Trans', '##actions'] , add_special_tokens=True)
# [101, 5230, 3327, 13809, 22946, 102] 
# tokenizer.decode(101) == '[ C L S ]'
# tokenizer.decode(102) == [ S E P ]'
```

In [15]:
tokenizer(['Real estate', 'Trans', '##actions'])

{'input_ids': [[101, 5230, 3327, 102], [101, 13809, 102], [101, 108, 108, 3721, 102]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 1]]}

In [22]:
tokenizer.encode("Real estate")

[101, 5230, 3327, 102]

In [41]:
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer.tokenize("Real estate Transactions")
# ['Real', 'estate', 'Trans', '##actions']
token_ids = tokenizer.convert_tokens_to_ids(['Real', 'estate', 'Trans', '##actions'])
print(token_ids)
tokenizer.encode(['Real', 'estate', 'Trans', '##actions'] , add_special_tokens=True)
# [101, 5230, 3327, 13809, 22946, 102] 
# tokenizer.decode(101) == '[ C L S ]'
# tokenizer.decode(102) == [ S E P ]'

[5230, 3327, 13809, 22946]


[101, 5230, 3327, 13809, 22946, 102]

### Is the output of `4.5.1 longformer` the same as `2.6.0 bert`?
#### The output shape seems different

In [100]:
config = AutoConfig.from_pretrained("allenai/longformer-base-4096")
config

LongformerConfig {
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sep_token_id": 2,
  "transformers_version": "4.5.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [93]:
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
config = AutoConfig.from_pretrained("allenai/longformer-base-4096")
config.attention_window = [1024 for _ in config.attention_window]
config.output_hidden_states = True
model = AutoModel.from_pretrained("allenai/longformer-base-4096", config=config)

In [94]:
tokenized = tokenizer("Real estate transactions", return_tensors="pt")
tokenized

{'input_ids': tensor([[    0, 17105,  2587,  5538,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [95]:
encodings = model(**tokenized)

In [96]:
encodings[0].shape

torch.Size([1, 5, 768])

In [97]:
encodings[1].shape

torch.Size([1, 768])

In [98]:
encodings[2][0].shape

torch.Size([1, 1024, 768])

In [72]:
# SAMPLE_TEXT = ' '.join(['Hello world! '] * 1000)  # long input document

# input_ids = torch.tensor(tokenizer.encode(SAMPLE_TEXT)).unsqueeze(0)  # batch of size 1

# attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device) # initialize to local attention
# attention_mask[:, [1, 4, 21,]] =  2  # Set global attention based on the task. For example,
#                                      # classification: the <s> token
#                                      # QA: question tokens

# # padding seqlen to the nearest multiple of 512. Needed for the 'sliding_chunks' attention
# input_ids, attention_mask = pad_to_window_size(
#         input_ids, attention_mask, config.attention_window[0], tokenizer.pad_token_id)

# output = model(input_ids, attention_mask=attention_mask)[0]

In [9]:
tokenizer.decode(102)

'[ S E P ]'

In [36]:
tokenizer.encode?

In [29]:
tokenizer.encode(('ĠTransactions', 'Ġ') )

[0, 649, 21402, 19163, 28929, 2, 2, 649, 21402, 2]

In [35]:
tokenizer.decode(0)

'<s>'

In [25]:
tokenizer.encode?

In [20]:
tokenizer(['why hello there amazonianas']).input_ids

[[0, 25800, 20760, 89, 524, 20524, 811, 281, 2]]

In [37]:
all_bert_outputs = model(tokenizer(['why hello there amazonianas, amazonianas'], return_tensors="pt").input_ids)

In [27]:
len(all_bert_outputs)

3

In [38]:
all_bert_outputs[0].shape

torch.Size([1, 14, 768])

In [39]:
all_bert_outputs[2][-1].shape

torch.Size([1, 512, 768])

In [24]:
model.forward?